In [1]:
import ast

# Sample data entries, each containing function definition, test cases, and expected results.
entries = [
    {
        "task_id": "HumanEval/0",
        "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    ...",
        "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n",
        "test": "assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\nassert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n...",
        "entry_point": "has_close_elements"
    },
    # More entries like HumanEval/1, HumanEval/2, etc.
]

# Define a helper to run each function and compare with expected results.
def run_tests(entry):
    # Prepare the function
    function_code = entry["prompt"] + entry["canonical_solution"]
    function_name = entry["entry_point"]
    
    # Dynamically define the function
    exec(function_code, globals())
    
    # Define candidate function as per the entry_point name
    candidate = globals()[function_name]
    
    # Prepare and execute tests
    test_code = entry["test"]
    try:
        # Run test cases
        exec(test_code, {"candidate": candidate})
        print(f"{entry['task_id']} passed all tests.")
    except AssertionError:
        print(f"{entry['task_id']} failed one or more tests.")




In [4]:
import pandas as pd

# Load the dataset
df = pd.read_csv("results-fill-missing/result-meta-llama-Llama-3.1-70B-Instruct.csv")

# Function to run each test case and determine if it passes or fails
def run_tests(row):
    function_code = row["prompt"] + "\n" + row["canonical_solution"]
    function_name = row["entry_point"]
    test_code = row["test"]

    # Define the function dynamically in the global scope
    exec(function_code, globals())
    candidate = globals().get(function_name)  # Access the function by name

    # Run the test cases and determine result
    try:
        exec(test_code, {"candidate": candidate})
        return "Passed"
    except AssertionError:
        return "Failed"
    except Exception as e:
        return f"Error: {str(e)}"

# Apply test function to each row and store result in a new column
df["result"] = df.apply(run_tests, axis=1)

# Count the number of successes and failures
success_count = (df["result"] == "Passed").sum()
failure_count = len(df) - success_count

print(f"Number of successful tests: {success_count}")
print(f"Number of failed tests: {failure_count}")

# Extract the generated code for each row
df["extracted_code"] = df["canonical_solution"]

# Save the DataFrame with test results and extracted code to a new CSV file
df.to_csv("test_results_with_code.csv", columns=["task_id", "result", "extracted_code"], index=False)

# Display summary
print("Detailed test results with extracted code saved to 'test_results_with_code.csv'")


Number of successful tests: 164
Number of failed tests: 0
Detailed test results with extracted code saved to 'test_results_with_code.csv'
